In [1]:
import pandas as pd
import requests, urllib.parse, json

In [2]:
df = pd.read_json('data/MQuAKE-T.json')

In [9]:
df["answers"] = df.apply(lambda x: [x["answer"]]+x["answer_alias"], axis=1)
df["knowledge"] = df.apply(lambda x: [{
    "triples": triple,
    "triples_labeled": x["orig"]["new_triples_labeled"][i],
    "single_hops": x["new_single_hops"][i],
} for i, triple in enumerate(x["orig"]["new_triples"])], axis=1)
df = df[["case_id", "questions", "answers", "knowledge"]]

In [14]:
df

,case_id,questions,answers,knowledge
0,1,[Who is the head of government in the location...,"[Bill de Blasio, Bill deBlasio, Warren Wilhelm...","[{'triples': ['Q1109582', 'P159', 'Q60'], 'tri..."
1,2,[Who is the head of state of the country that ...,"[Donald Trump, @realDonaldTrump, David Denniso...","[{'triples': ['Q578102', 'P27', 'Q30'], 'tripl..."
2,3,[Who is the head of government of the country ...,"[Donald Trump, @realDonaldTrump, David Denniso...","[{'triples': ['Q7306656', 'P27', 'Q30'], 'trip..."
3,4,[Who is the head of state of the country that ...,"[Donald Trump, @realDonaldTrump, David Denniso...","[{'triples': ['Q1689359', 'P495', 'Q30'], 'tri..."
4,5,[Who is the current head of state of the count...,"[Donald Trump, @realDonaldTrump, David Denniso...","[{'triples': ['Q14049', 'P27', 'Q30'], 'triple..."
...,...,...,...,...
1863,1864,[Who is the head of government of the country ...,"[Moon Jae-in, President Moon]","[{'triples': ['Q380118', 'P136', 'Q213665'], '..."
1864,1865,[Who is the head of government of the country ...,"[Shinzō Abe, Abe Shinzo, Abe Shinzō, Abe Shinz...","[{'triples': ['Q1142098', 'P136', 'Q131578'], ..."
1865,1866,[Who is the head of state of the country where...,"[Moon Jae-in, President Moon]","[{'triples': ['Q482847', 'P136', 'Q213665'], '..."
1866,1867,[Who is the head of government of the country ...,"[Donald Trump, @realDonaldTrump, David Denniso...","[{'triples': ['Q592358', 'P26', 'Q496437'], 't..."


In [17]:
def check_triples_validity(row):
    """
    Check if the triples are available now
    """
    for triple_dict in row["knowledge"]:
        subject_id = triple_dict["triples"][0]
        predicate_id = triple_dict["triples"][1]
        object_id = triple_dict["triples"][2]
        query = f"ASK {{ wd:{subject_id} wdt:{predicate_id} wd:{object_id} }}"
        url = 'https://query.wikidata.org/sparql?' + urllib.parse.urlencode({
              'format': 'json', 'query': query})
        if not requests.get(url).ok:
            return False
    return True

# Filter out cases where the triples are not available now
df = df[df.apply(check_triples_validity, axis=1)]

In [18]:
df

,case_id,questions,answers,knowledge
0,1,[Who is the head of government in the location...,"[Bill de Blasio, Bill deBlasio, Warren Wilhelm...","[{'triples': ['Q1109582', 'P159', 'Q60'], 'tri..."
1,2,[Who is the head of state of the country that ...,"[Donald Trump, @realDonaldTrump, David Denniso...","[{'triples': ['Q578102', 'P27', 'Q30'], 'tripl..."
2,3,[Who is the head of government of the country ...,"[Donald Trump, @realDonaldTrump, David Denniso...","[{'triples': ['Q7306656', 'P27', 'Q30'], 'trip..."
3,4,[Who is the head of state of the country that ...,"[Donald Trump, @realDonaldTrump, David Denniso...","[{'triples': ['Q1689359', 'P495', 'Q30'], 'tri..."
4,5,[Who is the current head of state of the count...,"[Donald Trump, @realDonaldTrump, David Denniso...","[{'triples': ['Q14049', 'P27', 'Q30'], 'triple..."
82,83,[Who is the head of state of the Langston Hugh...,"[Donald Trump, @realDonaldTrump, David Denniso...","[{'triples': ['Q188093', 'P27', 'Q30'], 'tripl..."
243,244,[Who is the head of government at the headquar...,"[Bill de Blasio, Bill deBlasio, Warren Wilhelm...","[{'triples': ['Q5061606', 'P159', 'Q60'], 'tri..."
1189,1190,[Who is the head of government of the country ...,[Saulius Skvernelis],"[{'triples': ['Q627388', 'P27', 'Q37'], 'tripl..."
